# Test with 1-D input

In [ ]:
import sys, os
# get the parent directory of this notebook
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from learning_loop.active_learning import active_learning_loop
# 
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize_scalar
from scipy.optimize import differential_evolution

In [31]:
# test function & its global minimum
def f(x):
    x = np.asarray(x)
    return (x - 0.65)**2 +  np.sin(x)
res = minimize_scalar(f, bounds=(0, 1), method="bounded")
if res.success:
    print(f"Global minimum found:")
    print(f"  x* = {res.x:.6f}")
    print(f"  f(x*) = {res.fun:.6f}")
else:
    print("Optimization failed!")

Global minimum found:
  x* = 0.156076
  f(x*) = 0.399404


In [45]:
# starting sample size = 1, with 1000 grid points for candidate selection
X0 = np.array([[0.5]])
Y0 = np.array([f(0.5)])
X_grid = np.linspace(0, 1, 1000).reshape(-1, 1)
# --- Configurable NN setup ---
nn_cfg = {
    "hidden_layers": [64, 32, 16], # list of hidden layer sizes
    "activation": "relu",     # or "relu", "sigmoid", "leakyrelu"
    "dropout": 0.1,
    "epochs": 200,
    "lr": 1e-3,
}
# --- Run the loop ---
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=50,
    model_type="nn",
    nn_config=nn_cfg,
    acquisition = "LCB", kappa_start=3.0, kappa_end=1.5,
    min_improve_pct=None
)
# --- Extract history ---
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])
best_y = np.array([h[3] for h in history])
best_final = np.min(best_y)
# plot the tuning convergence
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters+1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=res.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {res.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Neural Network Model)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white"
)
fig.show()


Starting active learning run — initial best_y = 0.5019
[001/50] acq=LCB, x_next=0.0000, y_next=0.4225 (new), best_y=0.4225
[002/50] acq=LCB, x_next=0.0751, y_next=0.4055 (new), best_y=0.4055
[003/50] acq=LCB, x_next=0.0010, y_next=0.4222 (new), best_y=0.4055
[004/50] acq=LCB, x_next=0.0661, y_next=0.4070 (new), best_y=0.4055
[005/50] acq=LCB, x_next=0.0020, y_next=0.4219 (new), best_y=0.4055
[006/50] acq=LCB, x_next=0.0030, y_next=0.4216 (new), best_y=0.4055
[007/50] acq=LCB, x_next=0.0040, y_next=0.4213 (new), best_y=0.4055
[008/50] acq=LCB, x_next=0.0050, y_next=0.4210 (new), best_y=0.4055
[009/50] acq=LCB, x_next=0.0671, y_next=0.4068 (new), best_y=0.4055
[010/50] acq=LCB, x_next=0.0310, y_next=0.4141 (new), best_y=0.4055
[011/50] acq=LCB, x_next=0.0060, y_next=0.4207 (new), best_y=0.4055
[012/50] acq=LCB, x_next=0.0070, y_next=0.4204 (new), best_y=0.4055
[013/50] acq=LCB, x_next=0.0080, y_next=0.4202 (new), best_y=0.4055
[014/50] acq=LCB, x_next=0.0090, y_next=0.4199 (new), best_y=

In [43]:
# starting sample size = 1, with 1000 grid points for candidate selection
X0 = np.array([[0.5]])
Y0 = np.array([f(0.5)])
X_grid = np.linspace(0, 1, 1000).reshape(-1, 1)
# --- Run the loop ---
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=50,
    model_type="poly",
    acquisition = "LCB", kappa_start=3.0, kappa_end=1.5,
    min_improve_pct=None
)
# --- Extract history ---
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])
best_y = np.array([h[3] for h in history])
best_final = np.min(best_y)
# plot the tuning convergence
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters+1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=res.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {res.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Polynomial Model)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white"
)
fig.show()


Starting active learning run — initial best_y = 0.5019
[001/50] acq=LCB, x_next=0.0000, y_next=0.4225 (new), best_y=0.4225
[002/50] acq=LCB, x_next=0.0010, y_next=0.4222 (new), best_y=0.4222
[003/50] acq=LCB, x_next=0.1872, y_next=0.4003 (new), best_y=0.4003
[004/50] acq=LCB, x_next=0.1552, y_next=0.3994 (new), best_y=0.3994
[005/50] acq=LCB, x_next=0.1562, y_next=0.3994 (new), best_y=0.3994
[006/50] acq=LCB, x_next=0.1572, y_next=0.3994 (new), best_y=0.3994
[007/50] acq=LCB, x_next=0.1542, y_next=0.3994 (new), best_y=0.3994
[008/50] acq=LCB, x_next=0.1582, y_next=0.3994 (new), best_y=0.3994
[009/50] acq=LCB, x_next=0.1532, y_next=0.3994 (new), best_y=0.3994
[010/50] acq=LCB, x_next=0.1592, y_next=0.3994 (new), best_y=0.3994
[011/50] acq=LCB, x_next=0.1522, y_next=0.3994 (new), best_y=0.3994
[012/50] acq=LCB, x_next=0.1602, y_next=0.3994 (new), best_y=0.3994
[013/50] acq=LCB, x_next=0.1512, y_next=0.3994 (new), best_y=0.3994
[014/50] acq=LCB, x_next=0.1612, y_next=0.3994 (new), best_y=

In [44]:
# starting sample size = 1, with 1000 grid points for candidate selection
X0 = np.array([[0.5]])
Y0 = np.array([f(0.5)])
X_grid = np.linspace(0, 1, 1000).reshape(-1, 1)
# --- Run the loop ---
X_obs, Y_obs, history, cache = active_learning_loop(
    f=f,
    X_obs=X0,
    Y_obs=Y0,
    X_grid=X_grid,
    n_iters=50,
    model_type="tree",
    acquisition = "LCB", kappa_start=3.0, kappa_end=1.5,
    min_improve_pct=None
)
# --- Extract history ---
iters = np.arange(len(history))
y_next = np.array([h[2] for h in history])
best_y = np.array([h[3] for h in history])
best_final = np.min(best_y)
# plot the tuning convergence
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=iters+1, y=y_next,
    mode='lines+markers',
    name='Y evaluated',
    line=dict(color='royalblue', width=2),
    marker=dict(size=6)
))
fig.add_hline(
    y=res.fun,
    line_dash='dot',
    line_color='green',
    annotation_text=f"Final Best Y = {res.fun:.4f}",
    annotation_position='top right'
)
fig.update_layout(
    title="Active Learning Convergence (Random Forest Model)",
    xaxis_title="Iteration number",
    yaxis_title="Objective value (Y)",
    hovermode="x unified",
    template="plotly_white"
)
fig.show()


Starting active learning run — initial best_y = 0.5019
[001/50] acq=LCB, x_next=0.0000, y_next=0.4225 (new), best_y=0.4225
[002/50] acq=LCB, x_next=0.0010, y_next=0.4222 (new), best_y=0.4222
[003/50] acq=LCB, x_next=0.0020, y_next=0.4219 (new), best_y=0.4219
[004/50] acq=LCB, x_next=0.0030, y_next=0.4216 (new), best_y=0.4216
[005/50] acq=LCB, x_next=0.0060, y_next=0.4207 (new), best_y=0.4207
[006/50] acq=LCB, x_next=0.0070, y_next=0.4204 (new), best_y=0.4204
[007/50] acq=LCB, x_next=0.0300, y_next=0.4144 (new), best_y=0.4144
[008/50] acq=LCB, x_next=0.0320, y_next=0.4139 (new), best_y=0.4139
[009/50] acq=LCB, x_next=0.0621, y_next=0.4077 (new), best_y=0.4077
[010/50] acq=LCB, x_next=0.0601, y_next=0.4081 (new), best_y=0.4077
[011/50] acq=LCB, x_next=0.0891, y_next=0.4036 (new), best_y=0.4036
[012/50] acq=LCB, x_next=0.2883, y_next=0.4151 (new), best_y=0.4036
[013/50] acq=LCB, x_next=0.0801, y_next=0.4048 (new), best_y=0.4036
[014/50] acq=LCB, x_next=0.0811, y_next=0.4047 (new), best_y=